# Import Lib

In [60]:
import pandas as pd 
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
from sklearn.linear_model import LinearRegression
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso,Ridge
from sklearn.model_selection import GridSearchCV, cross_val_score

warnings.filterwarnings(
    "ignore", category=DeprecationWarning
)  # to avoid deprecation warnings

import sys
sys.path.append(r'C:\Users\antoi\Documents\Work_Learn\JEDHA\M05-Supervised_ML\JEDHA-Projet')
from function import *

# EDA

In [61]:
df = pd.read_csv("Walmart_Store_sales.csv")
describe_df(df)

number of rows: 150
Display of dataset:


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Basic statistics:


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


Pourcentage of missing values:


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [62]:
df['Store'].value_counts()
df['Store'].unique()

array([ 6., 13., 17., 11.,  4., 15., 20., 14.,  3.,  8., 18.,  7.,  1.,
        2.,  5., 19., 16., 12.,  9., 10.])

In [63]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day_of_week'] = df['Date'].dt.day_of_week
df['day_of_year'] = df['Date'].dt.day_of_year

df = df.drop(['Date'],axis=1) 

C:\Users\antoi\AppData\Local\Temp\ipykernel_30812\1572134870.py:1: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [64]:
print('{} samples before'.format(df.shape[0]))

df.dropna(subset=['Weekly_Sales'],inplace=True)
df = df.reset_index(drop=True)

cols = ['Temperature', 'Fuel_Price', 'CPI','Unemployment']

for col in cols:
    col_mean = df[col].mean()
    col_std = df[col].std()
    
    lower_bound = col_mean - 3 * col_std
    upper_bound = col_mean + 3 * col_std
    df_filtered = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    outliers = df[(df[col] > upper_bound) | (df[col] < lower_bound)]
    if not outliers.empty:
        print(f"Outliers in column '{col}': {len(outliers)} rows")

df = df_filtered
print('{} samples after'.format(df.shape[0]))


150 samples before
Outliers in column 'Unemployment': 5 rows
117 samples after


In [65]:
# print(df['day_of_week'].value_counts())
# print(df['Holiday_Flag'].value_counts())

#On peut enlever ces colonnes qui ont qu'une seule valeur

target = 'Weekly_Sales'
remove_col = ['day_of_week','day_of_year', 'Holiday_Flag']
exclude_cols = [target] + remove_col

features = [col for col in df.columns if col not in exclude_cols]
categorical_features = ['Store']
numeric_features = [col for col in features if col not in categorical_features]

df.drop(remove_col,axis=1,inplace=True)

df.columns,features, numeric_features, categorical_features


(Index(['Store', 'Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI',
        'Unemployment', 'year', 'month'],
       dtype='object'),
 ['Store',
  'Temperature',
  'Fuel_Price',
  'CPI',
  'Unemployment',
  'year',
  'month'],
 ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'year', 'month'],
 ['Store'])

### Plot

In [66]:
fig = px.scatter(df,x="Unemployment",y="CPI")
fig.update_layout(bargap=0.1, xaxis=dict(tickmode='linear', dtick=1))
fig.show()

In [67]:
# fig = px.histogram(df,x="month",y="CPI",color= df["year"].astype(str).tolist(),nbins=12)
fig = px.histogram(df,x="Unemployment",y="CPI",color= df["year"].astype(str).tolist(),nbins=20)
fig.update_layout(bargap=0.1, xaxis=dict(tickmode='linear', dtick=1))
fig.show()

In [68]:
fig = px.histogram(df,x="Store",y="Weekly_Sales",color= df["year"].astype(str).tolist(),nbins=20)
fig.update_layout(bargap=0.1, xaxis=dict(tickmode='linear', dtick=1))
fig.show()

In [69]:
fig = px.histogram(df,x="month",y="Weekly_Sales",color= df["year"].astype(str).tolist(),nbins=12)
fig.update_layout(bargap=0.1)

fig.show()

In [70]:


import plotly.express as px
fig = px.histogram(df, x="year", nbins=len(df['year'].unique()))
fig.update_layout(bargap=0.1)
fig.show()


fig = px.histogram(df, x="month", nbins=12)
fig.update_layout(bargap=0.1)
fig.show()


fig = px.histogram(df, x="CPI",nbins=100)
fig.update_layout(bargap=0.1)
fig.show()

In [71]:
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1000, width = 1000)
fig.show()

In [72]:
corr_matrix = df.corr().round(2)
fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist()
                                  )

fig.update_layout(height=800, width = 800)

fig.show()

In [73]:
describe_df(df)

number of rows: 117
Display of dataset:


,Store,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,year,month
0,6.0,1572117.54,59.61,3.045,214.777523,6.858,2011.0,2.0
1,13.0,1807545.43,42.38,3.435,128.616064,7.470,2011.0,3.0
2,11.0,1244390.03,84.57,NaN,214.556497,7.346,NaN,NaN
3,6.0,1644470.66,78.89,2.759,212.412888,7.092,2010.0,5.0
4,4.0,1857533.70,NaN,2.756,126.160226,7.896,2010.0,5.0


Basic statistics:


,Store,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,year,month
count,117.000000,1.170000e+02,106.000000,106.000000,108.000000,117.000000,102.000000,102.000000
mean,10.000000,1.268911e+06,60.581792,3.338991,179.750523,7.399427,2010.882353,6.294118
std,6.247758,6.532565e+05,17.935627,0.475925,39.674311,0.994117,0.824070,3.129509
min,1.000000,2.689290e+05,18.790000,2.548000,126.126600,5.143000,2010.000000,1.000000
25%,4.000000,5.706112e+05,45.342500,2.850750,132.610242,6.664000,2010.000000,4.000000
50%,9.000000,1.420405e+06,61.450000,3.497500,197.500965,7.368000,2011.000000,6.000000
75%,16.000000,1.807545e+06,75.485000,3.737750,214.892987,8.099000,2012.000000,8.750000
max,20.000000,2.771397e+06,91.650000,4.170000,226.968844,9.524000,2012.000000,12.000000


Pourcentage of missing values:


Store            0.000000
Weekly_Sales     0.000000
Temperature      9.401709
Fuel_Price       9.401709
CPI              7.692308
Unemployment     0.000000
year            12.820513
month           12.820513
dtype: float64

In [74]:
df.nunique().sort_values()


year              3
month            12
Store            19
Fuel_Price       96
Unemployment     96
Temperature     105
CPI             107
Weekly_Sales    117
dtype: int64

# First Model

## V1

In [75]:
numeric_strategy="median"
categorical_strategy="most_frequent"

In [76]:
Y = df[target]
X = df[features]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=numeric_strategy)),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=categorical_strategy)),
        ("encoder", OneHotEncoder(drop="first")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test) 

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))

new_columns = list(preprocessor.transformers_[1][1]['encoder'].get_feature_names_out(categorical_features))
transformed_columns = numeric_features + new_columns
len(transformed_columns)

coefs = pd.DataFrame(index = transformed_columns, data = model.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients',ascending=False)

fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()



Accuracy on training set :  0.9686778975929555
Accuracy on test set :  0.9510325716474645


In [77]:
feature_importance

,coefficients
Store_5.0,1.273589e+06
Store_9.0,1.136645e+06
Store_3.0,1.106600e+06
Store_16.0,1.004634e+06
Store_7.0,9.296745e+05
Store_4.0,8.070672e+05
Store_8.0,6.811461e+05
Store_15.0,6.598461e+05
Store_14.0,6.166813e+05
Store_13.0,5.816516e+05


In [78]:
lasso = Lasso(max_iter=10000)

params = {
    'alpha': np.arange(1,2000)
}

grid_search = GridSearchCV(lasso, params, cv=5)
grid_search.fit(X_train, Y_train)

best_alpha = grid_search.best_params_['alpha']
best_lasso = grid_search.best_estimator_

best_lasso.fit(X_train, Y_train)

Y_train_pred = best_lasso.predict(X_train)
Y_test_pred = best_lasso.predict(X_test)

train_score = best_lasso.score(X_train, Y_train)
test_score = best_lasso.score(X_test, Y_test)
print("Best alpha:", best_alpha)
print("Accuracy on training set:", train_score)
print("Accuracy on test set:", test_score)


scores = cross_val_score(best_lasso, X_train, Y_train, cv=5)

# Print the mean and standard deviation of the scores
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())

coefs = pd.DataFrame(index = transformed_columns, data = model.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients',ascending=False)

fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

Best alpha: 915
Accuracy on training set: 0.9675169400788814
Accuracy on test set: 0.9418969109606689
Mean cross-validation score: 0.9268877546778811
Standard deviation of cross-validation scores: 0.028944805082358027


In [79]:
feature_importance.head().index.tolist()

['Store_5.0', 'Store_9.0', 'Store_3.0', 'Store_16.0', 'Store_7.0']

In [80]:
df1 = pd.get_dummies(df, columns=[ 'Store'], prefix=['Store'])

target = 'Weekly_Sales'
Y = df1[target]
X = df1.drop(target, axis=1)

numeric_features = []
categorical_features = []
for i, t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)):
        numeric_features.append(i)
    else:
        categorical_features.append(i)

print('Found numeric features:', numeric_features)
print('Found categorical features:', categorical_features)
display(df1.head())
df1[categorical_features] = df1[categorical_features].astype(int)
display(df1.columns)
display(df1.head())

# numeric_features += categorical_features
print(len(numeric_features))


Found numeric features: ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'year', 'month']
Found categorical features: ['Store_1.0', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0']


,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,year,month,Store_1.0,Store_2.0,Store_3.0,...,Store_10.0,Store_11.0,Store_13.0,Store_14.0,Store_15.0,Store_16.0,Store_17.0,Store_18.0,Store_19.0,Store_20.0
0,1572117.54,59.61,3.045,214.777523,6.858,2011.0,2.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1807545.43,42.38,3.435,128.616064,7.470,2011.0,3.0,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1244390.03,84.57,NaN,214.556497,7.346,NaN,NaN,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,1644470.66,78.89,2.759,212.412888,7.092,2010.0,5.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1857533.70,NaN,2.756,126.160226,7.896,2010.0,5.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Index(['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
       'year', 'month', 'Store_1.0', 'Store_2.0', 'Store_3.0', 'Store_4.0',
       'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0',
       'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0',
       'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0'],
      dtype='object')

,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,year,month,Store_1.0,Store_2.0,Store_3.0,...,Store_10.0,Store_11.0,Store_13.0,Store_14.0,Store_15.0,Store_16.0,Store_17.0,Store_18.0,Store_19.0,Store_20.0
0,1572117.54,59.61,3.045,214.777523,6.858,2011.0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1807545.43,42.38,3.435,128.616064,7.470,2011.0,3.0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1244390.03,84.57,NaN,214.556497,7.346,NaN,NaN,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1644470.66,78.89,2.759,212.412888,7.092,2010.0,5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1857533.70,NaN,2.756,126.160226,7.896,2010.0,5.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


6


In [81]:
corr_matrix = df1.corr().round(2)

corr_list = []
corr_list_flat = []

for c in corr_matrix.columns:
    for r, corr_val in corr_matrix[c].items():
        if (c != r) and abs(corr_val) >= 0.35:
                corr_list.append((c, r))
                corr_list_flat.append(c)
corr_list
corr_list_flat = np.unique(np.array(corr_list_flat))
df2 = df1[corr_list_flat]
corr_matrix2 = df2.corr().round(2)
df2



,CPI,Fuel_Price,Store_13.0,Store_18.0,Store_3.0,Store_5.0,Unemployment,Weekly_Sales,year
0,214.777523,3.045,0,0,0,0,6.858,1572117.54,2011.0
1,128.616064,3.435,1,0,0,0,7.470,1807545.43,2011.0
2,214.556497,NaN,0,0,0,0,7.346,1244390.03,NaN
3,212.412888,2.759,0,0,0,0,7.092,1644470.66,2010.0
4,126.160226,2.756,0,0,0,0,7.896,1857533.70,2010.0
...,...,...,...,...,...,...,...,...,...
128,NaN,3.285,0,0,1,0,7.197,403342.40,2011.0
129,214.495838,2.705,0,0,1,0,7.343,396968.80,2010.0
130,226.968844,3.594,0,0,1,0,6.034,424513.08,2012.0
131,182.442420,2.780,0,0,0,0,8.899,2248645.59,2010.0


In [82]:
corr_matrix = df2.corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist()
                                  )

fig.update_layout(height=800, width = 800)

fig.show()

In [83]:
corr_matrix = df2.corr().round(2)
weekly_sales_corr = corr_matrix['Weekly_Sales'].drop('Weekly_Sales') 
fig = ff.create_annotated_heatmap(weekly_sales_corr.values.reshape(1, -1),
                                  x=weekly_sales_corr.index.tolist(),
                                  y=['Weekly_Sales'],
                                  colorscale='Viridis',
                                  showscale=True)

fig.update_layout(title='Correlation of Weekly_Sales with other features',
                  height=400,
                  width=1200)
fig.show()

In [84]:
Y = df[target]
features_v1 = ['Store_13.0','Store_5.0','Store_3.0']
X = df2[features_v1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=numeric_strategy)),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer,features_v1),
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test) 

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))

coefs = pd.DataFrame(index = features_v1, data = model.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients',ascending=False)

fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

Accuracy on training set :  0.40410266566142616
Accuracy on test set :  0.4489832701477543


# testing

In [85]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
import numpy as np

# Extract the top 5 features from the feature_importance DataFrame
top_features = feature_importance.head().index.tolist()

# Select only the top 5 features from X_train and X_test
X_train_top = X_train[:, [transformed_columns.index(feature) for feature in top_features]]
X_test_top = X_test[:, [transformed_columns.index(feature) for feature in top_features]]

ridge = Ridge(max_iter=10000)

params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
}
grid_search = GridSearchCV(ridge, params, cv=5)
grid_search.fit(X_train, Y_train)

best_alpha_ridge = grid_search.best_params_['alpha']
best_ridge = grid_search.best_estimator_

best_ridge.fit(X_train, Y_train)

Y_train_pred_ridge = best_ridge.predict(X_train)
Y_test_pred_ridge = best_ridge.predict(X_test)

train_score_ridge = best_ridge.score(X_train, Y_train)
test_score_ridge = best_ridge.score(X_test, Y_test)
print("Best alpha for Ridge:", best_alpha_ridge)
print("Accuracy on training set (Ridge):", train_score_ridge)
print("Accuracy on test set (Ridge):", test_score_ridge)

scores_ridge = cross_val_score(best_ridge, X_train, Y_train, cv=5)

print("Mean cross-validation score (Ridge):", scores_ridge.mean())
print("Standard deviation of cross-validation scores (Ridge):", scores_ridge.std())

IndexError: index 7 is out of bounds for axis 1 with size 3

- The accuracy on the training set (0.9732) is high, indicating that the model fits the training data well.

- The accuracy on the test set (0.9283) is also high, suggesting that the model generalizes well to unseen data.

- The mean cross-validation score (0.9412) is close to the test set accuracy, indicating that the model performs consistently well across different subsets of the training data.

## V2

# Second model feature eng 

## V1

In [ ]:
X.columns

Index(['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'year', 'month',
       'Store_1.0', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0',
       'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0',
       'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0',
       'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0'],
      dtype='object')

In [ ]:
target = 'Weekly_Sales'
Y = df[target]
X = df[features]


numeric_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("poly_features_2", PolynomialFeatures(degree=2)),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder",OneHotEncoder(drop="first"),),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))
# model.coef_

ValueError: A given column is not a column of the dataframe

## V2

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Lasso,Ridge

from sklearn.preprocessing import PolynomialFeatures

target = 'Weekly_Sales'


numeric_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder",OneHotEncoder(drop="first"),),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

regressor = Ridge()

params = {
    'alpha': np.arange(1, 100,1)
}

best_ridge = GridSearchCV(regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
best_ridge.fit(X_train, Y_train)
print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Lasso,Ridge

from sklearn.preprocessing import PolynomialFeatures

target = 'Weekly_Sales'


numeric_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder",OneHotEncoder(drop="first"),),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

regressor = Lasso()

params = {
    'alpha': [1, 2, 3, 5, 10, 20, 30]
}

best_ridge = GridSearchCV(regressor, param_grid = params, cv = 3)
best_ridge.fit(X_train, Y_train)
print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)